In [0]:
movies_df = spark.table("az_adb_simbus_training.imdb_project_schema.bronze_title_basics")
movies_df = (movies_df.select('tconst','titleType','primaryTitle').filter('titleType = "movie"')).na.drop(how='any')
# display(movies_df)

In [0]:
actor_or_director_df = spark.table("az_adb_simbus_training.imdb_project_schema.bronze_title_principals")
actor_or_director_df = (actor_or_director_df.select('tconst','category').filter('category = "actor" or category = "director"')).na.drop(how='any')
display(actor_or_director_df)